In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from collections import Counter
import copy

## hexadecimal generator

In [2]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [10]:
## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
# one individual to change to multiple
for individual in individuals_sum_dup_no_self_loop:
#     try:
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    ## preprocessing
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
#     except:
#         print(individual)
#         print(edges)
        
#         continue
    try:
        mid_to_weight(dic)
    except:
        print('mid_to_weight', individual)
        print(edges)
        print()
        continue   
    try:    
        out_to_weight(dic)
    except:
        print('out_to_weight', individual)
        print(edges)
        print()    

out_to_weight 2
[('mid0', 'mid1', 2.602655272394323), ('mid0', 'out2', -1.8113230581413093), ('mid1', 'out0', 1.7328246604608577), ('in1', 'mid1', 3.0427590416603083), ('mid0', 'out0', -1.9767434762704106), ('mid1', 'mid1', -2.559560506638181), ('in2', 'out1', 2.1034640622615597), ('in0', 'out2', 1.4107736914390356), ('in2', 'mid1', -1.4333587669769572)]



In [19]:
individuals_sum_dup_no_self_loop[2]

{'0mid1mid': ['mid0', 'mid1', 2.602655272394323],
 '0mid2out': ['mid0', 'out2', -1.8113230581413093],
 '1mid0out': ['mid1', 'out0', 1.7328246604608577],
 '1in1mid': ['in1', 'mid1', 3.0427590416603083],
 '0mid0out': ['mid0', 'out0', -1.9767434762704106],
 '1mid1mid': ['mid1', 'mid1', -2.559560506638181],
 '2in1out': ['in2', 'out1', 2.1034640622615597],
 '0in2out': ['in0', 'out2', 1.4107736914390356],
 '2in1mid': ['in2', 'mid1', -1.4333587669769572]}

In [20]:
edges = individuals_sum_dup_no_self_loop[2]
edges = [tuple(edges[i]) for i in edges]
edges

[('mid0', 'mid1', 2.602655272394323),
 ('mid0', 'out2', -1.8113230581413093),
 ('mid1', 'out0', 1.7328246604608577),
 ('in1', 'mid1', 3.0427590416603083),
 ('mid0', 'out0', -1.9767434762704106),
 ('mid1', 'mid1', -2.559560506638181),
 ('in2', 'out1', 2.1034640622615597),
 ('in0', 'out2', 1.4107736914390356),
 ('in2', 'mid1', -1.4333587669769572)]

In [21]:
dic = weight_sum_preprocessing(edges)
dic

{'mid1': {'mid0': 2.602655272394323,
  'in1': 3.0427590416603083,
  'mid1': -2.559560506638181,
  'in2': -1.4333587669769572},
 'out0': {'mid1': 1.7328246604608577, 'mid0': -1.9767434762704106},
 'out1': {'in2': 2.1034640622615597},
 'out2': {'mid0': -1.8113230581413093, 'in0': 1.4107736914390356}}

In [26]:
def mid_to_weight(dic):
    '''weight for clear input ie. no nested weights: 0mid:{0in, 1in} or 0mid:{0mid, 0in}'''
    for key in dic:
        
        if 'mid' in key and isinstance(dic[key], dict):
#             print(key)
            if set(i[:2] for i in dic[key]) == set(['in']):
                
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
                
            elif key in dic[key].keys() and Counter(i[:2] for i in dic[key])['mi'] == 1:
                
#                 print(dic[key])
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
mid_to_weight(dic)
dic

{'mid1': {'mid0': 2.602655272394323,
  'in1': 3.0427590416603083,
  'mid1': -2.559560506638181,
  'in2': -1.4333587669769572},
 'out0': {'mid1': 1.7328246604608577, 'mid0': -1.9767434762704106},
 'out1': {'in2': 2.1034640622615597},
 'out2': {'mid0': -1.8113230581413093, 'in0': 1.4107736914390356}}

In [32]:
def out_to_weight(dic):
    '''weight for nested input ie.  0mid:{1mid} or out:{0mid, 0in}
    use 'mid' neuron only if feeded with 'in' neurons'''
    for key in dic:
        if 'mid' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if dic[sub_key] and 'mid' in sub_key and isinstance(dic[sub_key], float):
#                     print(sub_key, dic[sub_key])
                    dic[key][sub_key] += dic[sub_key]

            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
                
        elif 'out' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key and sub_key in dic:
                    dic[key][sub_key] += dic[sub_key]
            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
            
out_to_weight(dic)
dic

KeyError: 'mid0'

In [30]:
dic[sub_key]

NameError: name 'sub_key' is not defined

In [14]:
mid_to_weight(dic)
dic

{'mid1': -0.9509717816997041,
 'out0': -0.36316591991614544,
 'out1': -0.7750813925109867,
 'out2': 0.9971658131325868}

In [ ]:
dic

In [8]:
edges = [('in1', 'out0', 1.4279871814436136), ('in0', 'out0', -3.0119945063329774), ('mid1', 'out0', -1.1555012971158247), ('in1', 'mid1', -1.2089729894704715), ('mid1', 'out2', -2.6342743781474134), ('mid0', 'out0', -3.9923088661681674)]